In [17]:
pip install vk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import vk
from vk.session import requests
import json

In [19]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [20]:
import copy

In [21]:
from google.colab import files
FileRead = open('/content/token.txt', 'r')
token = FileRead.read()#Вывод первых 100 символов
# в конце файл необходимо закрыть
FileRead.close()
version = 5.126
users_idGroup = [181948702, 154308430, 152087036, 143779216, 185215592, 161594007, 472583599, 113001678, 183648122, 196404046, 
    85770676, 355133495, 137872152, 102711908, 126568936, 507184348, 458357505, 131765293, 323208790, 173792332, 343410069, 
    346034855, 255053910, 338686058, 312616596]

In [22]:
def RequestsUsersGet(users_id, i):
  response = requests.get("https://api.vk.com/method/users.get",
                        params={
                            "user_id": users_id[i],
                            "name_case": "nom",
                            "fields": "nickname",
                            "access_token": token,
                            "v": 5.131
                        }
                        )  
  if(response.json()):
    data = response.json()
  return data

In [23]:
def RequestsFriendsGet(users_id, j):
  response = requests.get("https://api.vk.com/method/friends.get",
                        params={
                            "user_id": users_id[j],
                            "order": "name",
                            "count": 5000,
                            "offset": 0,
                            "fields": "nickname",

                            "access_token": token,
                            "v": version
                        }
                        )
  if(response.json()):
    data = response.json()
  return data

In [ ]:
frameGroup = pd.DataFrame(columns=['main_id', 'first_name', 'last_name'])
frameFriends = pd.DataFrame(columns=['main_id', 'id_friend', 'first_name', 'last_name'])
frameFriendsOfFriends = pd.DataFrame(columns=['main_id', 'id_friend', 'first_name', 'last_name'])
GraphCentrality = nx.Graph()

#1 уровень (получаем информацию о одногруппниках)
for i in range(len(users_idGroup)):
  dataGroup = RequestsUsersGet(users_idGroup, i)
  try: 
    if dataGroup['response'][0]['is_closed'] == True: ClosedAcc = "Доступ к странице пользователя недоступен"
    else: frameGroup = frameGroup.append({'main_id': dataGroup['response'][0]['id'], 'first_name': dataGroup['response'][0]['first_name'], 'last_name': dataGroup['response'][0]['last_name']}, ignore_index=True)
  except:
    errorVK = "Ошибка сервера ВК - не получен доступ к аккаунту"

#2 уровень (получаем информациою о друзьях одногруппников)
for j in range(len(frameGroup.main_id)):
  dataFriends = RequestsFriendsGet(frameGroup.main_id, j)
  countOfUserFriends = dataFriends['response']['count']
  for jx in range(countOfUserFriends):
    try:
      if dataFriends['response']['items'][jx]['deactivated'] == "banned": banned = "Сработало исключение - banned"
      if dataFriends['response']['items'][jx]['deactivated'] == "deleted": deleted = "Сработало исключение - deleted"
    except:  
      try: 
        if dataFriends['response']['items'][jx]['is_closed'] == True: ClosedAcc = "Доступ к странице пользователя недоступен"
        else: 
          frameFriends = frameFriends.append({'main_id': frameGroup.main_id[j],'id_friend': dataFriends['response']['items'][jx]['id'], 'first_name': dataFriends['response']['items'][jx]['first_name'], 'last_name': dataFriends['response']['items'][jx]['last_name']}, ignore_index=True)
          GraphCentrality.add_edge(frameGroup.first_name[j] + " " + frameGroup.last_name[j], dataFriends['response']['items'][jx]['first_name'] + " " + dataFriends['response']['items'][jx]['last_name'])
      except: errorVK = "Ошибка сервера ВК - не получен доступ к аккаунту"


#2.1 проверка на связь друзей 2 уровня
#for n in range(len(frameFriends.id_friend.unique())):
  #ndataFriends = RequestsFriendsGet(frameFriends.id_friend.unique(), n)
  #ncountOfUserFriends = ndataFriends['response']['count']
  #for nx in range(ndataFriends['response']['count']): 
    #try:
      #if ndataFriends['response']['items'][nx]['deactivated'] == "banned": banned = "Сработало исключение - banned"
      #if ndataFriends['response']['items'][nx]['deactivated'] == "deleted": deleted = "Сработало исключение - deleted"
    #except:  
      #try: 
        #if ndataFriends['response']['items'][nx]['is_closed'] == True: ClosedAcc = "Доступ к странице пользователя недоступен"
        #else: 
          #for ny in range(len(frameFriends.id_friend)):
            #if ndataFriends['response']['items'][nx]['id'] == frameFriends.id_friend[ny]:
              #GraphCentrality.add_edge(frameFriends.first_name[ny] + " " + frameFriends.last_name[ny], ndataFriends['response']['items'][nx]['first_name'] + " " + ndataFriends['response']['items'][nx]['last_name'])
              #break
      #except: errorVK = "Ошибка сервера ВК - не получен доступ к аккаунту"

#3 уровень (получаем информацию о друзьях друзей одногруппников)
#for k in range(len(frameFriends.main_id)):
  #dataFriendsOfFrineds = RequestsFriendsGet(frameFriends.main_id, k)
  #countOfUserFriendsOfFriends = dataFriendsOfFrineds['response']['count']
  #for kx in range(countOfUserFriendsOfFriends):
    #try:
      #if dataFriendsOfFrineds['response']['items'][kx]['deactivated'] == "banned": banned = "Сработало исключение - banned"
      #if dataFriendsOfFrineds['response']['items'][kx]['deactivated'] == "deleted": deleted = "Сработало исключение - deleted"
    #except:  
      #try: 
        #if dataFriendsOfFrineds['response']['items'][kx]['is_closed'] == True: ClosedAcc = "Доступ к странице пользователя недоступен"
        #else: 
          #frameFriendsOfFriends = frameFriendsOfFriends.append({'main_id': frameFriends.main_id[j],'id_friend': dataFriendsOfFrineds['response']['items'][jx]['id'], 'first_name': dataFriendsOfFrineds['response']['items'][jx]['first_name'], 'last_name': dataFriendsOfFrineds['response']['items'][jx]['last_name']}, ignore_index=True)
          #GraphCentrality.add_edge(frameFriends.first_name[k] + " " + frameFriends.last_name[k], dataFriends['response']['items'][kx]['first_name'] + " " + dataFriends['response']['items'][kx]['last_name'])
      #except: errorVK = "Ошибка сервера ВК - не получен доступ к аккаунту"


#3.1 проверка на связь друзей 3 уровня
#for m in range(len(frameFriendsOfFriends.id_friend.unique())):
  #mdataFriendOfFriends = RequestsFriendsGet(frameFriendsOfFriends.id_friend.unique(), m)
  #mcountOfUserFriends = mdataFriendOfFriends['response']['count']
  #for mx in range(mcountOfUserFriends): 
    #try:
      #if mdataFriendOfFriends['response']['items'][mx]['deactivated'] == "banned": banned = "Сработало исключение - banned"
      #if mdataFriendOfFriends['response']['items'][mx]['deactivated'] == "deleted": deleted = "Сработало исключение - deleted"
    #except:  
      #try: 
        #if mdataFriendOfFriends['response']['items'][mx]['is_closed'] == True: ClosedAcc = "Доступ к странице пользователя недоступен"
        #else: 
          #for my in range(len(frameFriendsOfFriends.id_friend)):
            #if mdataFriendOfFriends['response']['items'][mx]['id'] == frameFriendsOfFriends.id_friend[my]:
              #GraphCentrality.add_edge(frameFriendsOfFriends.first_name[my] + " " + frameFriendsOfFriends.last_name[my], mdataFriendOfFriends['response']['items'][mx]['first_name'] + " " + mdataFriendOfFriends['response']['items'][mx]['last_name'])
              #break
      #except: errorVK = "Ошибка сервера ВК - не получен доступ к аккаунту"

In [25]:
pos = nx.spring_layout(GraphCentrality,scale=40, k=0.30, iterations=20)
nx.draw(GraphCentrality, pos, font_size=6, node_size=750, alpha=0.7, with_labels=True, edge_color='gray')
nx.draw(GraphCentrality)

Output hidden; open in https://colab.research.google.com to view.

In [26]:
close_centrality = nx.closeness_centrality(GraphCentrality)
print("Оценка центральности по посредничеству")
print(close_centrality)

Оценка центральности по посредничеству
{'Andrey Maslov': 0.39889150151158886, 'ピンクの雨 カップ・カップ': 0.2851825168107589, '叫张 志': 0.2851825168107589, 'Alexander Gerasimov': 0.2851825168107589, 'Alexander Rykalin': 0.2851825168107589, 'Alexander Tsvetkov': 0.2851825168107589, 'Alexandra Podnebesnova': 0.2851825168107589, 'Alexandra Yaremenko': 0.2851825168107589, 'Alexey Mizenin': 0.2851825168107589, 'Alexey Semendyaev': 0.2956554213867795, 'Alina Sativa': 0.2851825168107589, 'Alsu Khabibullova': 0.2851825168107589, 'Alyona Maslova': 0.2851825168107589, 'Alyona Vinogradova': 0.2851825168107589, 'Anastasia Fedorova': 0.29665251061703724, 'Anastasia Goncharova-Andreeva': 0.2851825168107589, 'Anastasia Khaleeva': 0.2851825168107589, 'Anastasia Morozova': 0.2851825168107589, 'Anatoly Maslov': 0.2851825168107589, 'Andrey Forshakov': 0.2956554213867795, 'Andrey Mekhanikov': 0.2851825168107589, 'Angelina Koryakina': 0.2851825168107589, 'Anna Galitskaya': 0.2851825168107589, 'Anna Novikova': 0.2851825

In [27]:
bet_centrality = nx.betweenness_centrality(GraphCentrality, normalized = True, endpoints = False)
print("Оценка центральности по близости")
print(bet_centrality)

Оценка центральности по близости
{'Andrey Maslov': 0.24961883108210337, 'ピンクの雨 カップ・カップ': 0.0, '叫张 志': 0.0, 'Alexander Gerasimov': 0.0, 'Alexander Rykalin': 0.0, 'Alexander Tsvetkov': 0.0, 'Alexandra Podnebesnova': 0.0, 'Alexandra Yaremenko': 0.0, 'Alexey Mizenin': 0.0, 'Alexey Semendyaev': 0.0, 'Alina Sativa': 0.0, 'Alsu Khabibullova': 0.0, 'Alyona Maslova': 0.0, 'Alyona Vinogradova': 0.0, 'Anastasia Fedorova': 0.0, 'Anastasia Goncharova-Andreeva': 0.0, 'Anastasia Khaleeva': 0.0, 'Anastasia Morozova': 0.0, 'Anatoly Maslov': 0.0, 'Andrey Forshakov': 0.0, 'Andrey Mekhanikov': 0.0, 'Angelina Koryakina': 0.0, 'Anna Galitskaya': 0.0, 'Anna Novikova': 0.0, 'Anna Zaytseva': 0.01001596999919004, 'Arina Apelgants': 0.0, 'Arsen Zinchenko': 0.0, 'Artem Khachatryan': 0.0, 'Artem Kurnenkov': 0.0, 'Artem Neizvestny': 0.0, 'Artem Novikov': 0.042913197103494206, 'Artem Shimanov': 0.0, 'Artemy Kiselev': 0.0, 'Artur Makushev': 0.0, 'Artyom Popov': 0.0, 'Berta Lunka': 0.0, 'Bogorog Vsemoguschy': 0.0, 'Da

In [28]:
eigenvector_centrality = nx.eigenvector_centrality_numpy(GraphCentrality)
print("Оценка центральности по собственному вектору")
print(eigenvector_centrality)

Оценка центральности по собственному вектору
{'Andrey Maslov': 0.002603425835430718, 'ピンクの雨 カップ・カップ': 7.886847599554195e-05, '叫张 志': 7.886847599554664e-05, 'Alexander Gerasimov': 7.886847599554698e-05, 'Alexander Rykalin': 7.886847599554061e-05, 'Alexander Tsvetkov': 7.886847599554325e-05, 'Alexandra Podnebesnova': 7.886847599553467e-05, 'Alexandra Yaremenko': 7.886847599554241e-05, 'Alexey Mizenin': 7.886847599553882e-05, 'Alexey Semendyaev': 0.00022874379451734677, 'Alina Sativa': 7.886847599553635e-05, 'Alsu Khabibullova': 7.886847599553412e-05, 'Alyona Maslova': 7.886847599554531e-05, 'Alyona Vinogradova': 7.886847599554592e-05, 'Anastasia Fedorova': 0.00025957206717982357, 'Anastasia Goncharova-Andreeva': 7.886847599553775e-05, 'Anastasia Khaleeva': 7.886847599554354e-05, 'Anastasia Morozova': 7.88684759955414e-05, 'Anatoly Maslov': 7.886847599554207e-05, 'Andrey Forshakov': 0.00022874379451733986, 'Andrey Mekhanikov': 7.886847599554686e-05, 'Angelina Koryakina': 7.886847599553858